# PROYECTO INTEGRADOR

# ANÁLISIS DE SENTIMIENTOS EN RESEÑAS DE PROFESORES

# NOTEBOOK 1: PREPROCESAMIENTO DE LOS DATOS 

## Cargar los datos

Se carga el dataset de reseñas de profesores que se utilizará en el proyecto. Este está en formato **TSV**.

- StudentComments: texto de la reseña.
- Rating: calificación del profesor, va de 0 a 5.
- Totalwords: cantidad de palabras de la reseña.
- Sentiment: sentimiento analizado por el modelo (positive, neutral, negative).
- Sent_pretrained: reseña objetiva o subjetiva (objective, subjective).
- Subjectivity: calificación de qué tan subjetiva es la reseña, va de 0 a 1.
- isSame: verifica que la entrada no esté duplicada.


In [11]:
import pandas as pd

path = "/kaggle/input/dataset-proyecto-integrador/ReadyToTrain_data_2col_with_subjectivity_final.tsv"
df = pd.read_table(path)

print(df.head())

   Unnamed: 0                                    StudentComments  Rating  \
0           0                                               good    4.96   
1           1                                               good    5.00   
2           2                                            teacher    4.25   
3           3  friendly teacher but not enough ability to enc...    4.38   
4           4                                            teacher    4.92   

   totalwords Sentiment sent_pretrained subjectivity  subj-score isSame  
0           1  positive        positive   subjective         0.6   true  
1           1  positive        positive   subjective         0.6   true  
2           1  positive         neutral    objective         0.0   fake  
3          10  positive         neutral   subjective         0.5   fake  
4           1  positive         neutral    objective         0.0   fake  


In [12]:
print(f"Cantidad de entradas: {len(df)}")

Cantidad de entradas: 2007747


## Eliminar columnas irrelevantes

El dataset incluye múltiples columnas que abordan distintos aspectos de las reseñas. No obstante, para los fines específicos del **análisis de sentimientos** que se llevará a cabo en este proyecto, varias de estas columnas no resultan relevantes. Por ello, se procederá a eliminarlas con el fin de optimizar los modelados.

Las variables que se conservarán serán el **texto de la reseña** y el **sentimiento** (positive, negative, neutral), puesto que estas son aquellas que utilizan los modelos de análisis de sentimientos al ser entrenados.

In [13]:
df = df[["StudentComments","Sentiment"]].copy()
print(df.head())

                                     StudentComments Sentiment
0                                               good  positive
1                                               good  positive
2                                            teacher  positive
3  friendly teacher but not enough ability to enc...  positive
4                                            teacher  positive


## Duplicados y valores nulos

Se eliminan todos los registros **duplicados** en la columna StudenComments.

In [14]:
df = df.drop_duplicates(subset="StudentComments").reset_index(drop=True)
print(df.head())

                                     StudentComments Sentiment
0                                               good  positive
1                                            teacher  positive
2  friendly teacher but not enough ability to enc...  positive
3                               he is a good techer.  positive
4                                he is agood techer.  positive


Se revisa, además, que no haya valores **nulos** en el dataset. Afortunadamente no los hay, por lo que no es necesario hacer eliminación o imputación de registros.

In [15]:
print(df.isnull().sum())

StudentComments    0
Sentiment          0
dtype: int64


In [16]:
print(f"Cantidad de entradas: {len(df)}")

Cantidad de entradas: 381005


## Limpieza del texto

- Convierte todo el texto a minúsculas.
- Elimina emojis.
- Elimina etiquetas HTML.
- Elimina caracteres no ASCII (acentos y símbolos raros).
- Reemplaza espacios consecutivos por uno solo.
- Elimina espacios innecesarios al inicio y al final.

In [17]:
import re

def clean_text(text):
    text = text.lower()
    
    text = re.sub(r'<[^>]+>', ' ', text)
    
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    text = re.sub(r"[^a-z0-9\s.,!?'-]", ' ', text)
    
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text.strip()

df["StudentComments"] = df["StudentComments"].astype(str).apply(clean_text)

print(df.head())


                                     StudentComments Sentiment
0                                               good  positive
1                                            teacher  positive
2  friendly teacher but not enough ability to enc...  positive
3                               he is a good techer.  positive
4                                he is agood techer.  positive


## Numerizar columna de sentimientos

En esta parte del proyecto es importante recalcar que el análisis de sentimientos que será realizado será **binario**, debido a que esta es la metodología que se suele utilizar en EAFIT. Por lo tanto, se dejará la clase "positive" como aquella positiva, y la "neutral" y "negative" serán aquella negativa.

Las palabras "positive" se cambiarán por "1", y "neutral" y "negative" se cambiarán por "0". Es necesario que los labels estén en formato numérico para el entrenamiento de los modelos.

In [18]:
df['Sentiment'] = df['Sentiment'].map({'positive': 1, 'neutral': 0, 'negative': 0})

print(df)

                                          StudentComments  Sentiment
0                                                    good          1
1                                                 teacher          1
2       friendly teacher but not enough ability to enc...          1
3                                    he is a good techer.          1
4                                     he is agood techer.          1
...                                                   ...        ...
381000          she is a great mentor and a great person.          1
381001              super friendly faculty i've ever seen          1
381002  tamanna nazneen rahman is an excellent teacher...          1
381003  best faculty i've ever seen. lucky to have him...          1
381004           data warehousing and data mining msccs a          0

[381005 rows x 2 columns]


## Verificar desbalanceo de clases

Tras haber llevado a cabo los distintos pasos del preprocesamiento, verificamos una vez más si el dataset sigue siendo **desbalanceado** (puesto que estaba muy desbalanceado inicialmente). De este modo podemos saber cuáles serán las mejores métricas para evaluar los modelos más adelante.

In [19]:
conteo = df['Sentiment'].value_counts().sort_index()

porcentajes = df['Sentiment'].value_counts(normalize=True).sort_index() * 100

resumen = pd.DataFrame({
    'Cantidad': conteo,
    'Porcentaje (%)': porcentajes.round(2)
})

print(resumen)

           Cantidad  Porcentaje (%)
Sentiment                          
0             72839           19.12
1            308166           80.88


Podemos observar que el dataset sigue siendo **desbalanceado**, ya que las reseñas positivas (categoría 1) representan aproximadamente el 80% de las entradas, mientras que las negativas (categoría 0) constituyen solo el 20%. Debido a este desequilibrio, NO es recomendable utilizar la accuracy como métrica principal, ya que podría dar una falsa impresión de buen desempeño al favorecer la clase mayoritaria. En su lugar, es preferible utilizar el **F1-Score**, ya que combina precisión y recall, y ofrece una evaluación más equilibrada del rendimiento del modelo para ambas clases.

## Preprocesamiento específico para modelos clásicos

En el proyecto se trabajará con dos tipos de modelos:

- **Modelo clásico**: basados en técnicas tradicionales como regresión logística o SVM. Se hará un **TF-IDF**.

- **Modelo de deep learning**: usando transformers que procesan el texto casi “crudo” con una limpieza básica y tokenización especial. Se hará un **DistilBERT**.

En este notebook solo se realizará el preprocesamiento común necesario para ambos enfoques, de modo que se pueda utilizar un dataset final único en el resto del proyecto. *Los pasos adicionales específicos para el modelo clásico, como vectorización y eliminación de stopwords, estarán disponibles en el notebook dedicado a ese modelo.*



## Descargar dataset limpio

Ahora se procede a descargar el dataset con todos los pasos de preprocesamiento aplicados. Este dataset se descargará en formato **CSV** puesto que se nos hace más cómodo trabajar con él.

In [20]:
df.to_csv("dataset_final.csv", index=False)